In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import math

C:\Users\Rohit\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
np.random.seed(100) 
pd.set_option('display.max_columns',10)
pd.set_option('display.width',None)
## Betas, Bootstraps are variable
bvec=[-0.1,0,0.1,0.2]
T=100
nsim = 1000
b0 = 1
boots=[10,20,30,50]
sig=0.5
mu_x=1
count = 0
outp =np.zeros((len(bvec),len(boots),6))
outp.shape
for bdx,b1 in enumerate(bvec):      
    for btx,boot in enumerate(boots):
        reject_boot = 0
        reject_both = 0
        reject_orig=0
        for isim in range(nsim):
            count+=1
            e = np.random.normal(0,sig,size=(T,1))
            x = np.random.normal(mu_x,sig,size=(T,1))
            y=b0+(b1*x)+e
            x = sm.add_constant(x)
            model_orig=smf.OLS(y,x).fit()
            # Create confindence interval for b1
            ci=np.zeros(int(boot))
            for b in range(boot):  ## Bootstrapping to find CI
                # Create a resample index
                resamp=np.random.randint(T, size=int(0.5*T)) #Re-sampling with replacement
                model = smf.OLS(y[resamp],x[resamp]).fit()
                ci[b] = model.params[1]
                #print(model.params[1])
            ci.sort()
            ci_lower = ci[int(boot*0.025)]
            ci_upper = ci[int(boot*0.975)]
            ## Empirical t-tests - Reject Null when CI follows a trend
            if not(0>=ci_lower and 0<=ci_upper): reject_boot+=1
            if not(0>=ci_lower and 0<=ci_upper) and model_orig.pvalues[1]<0.05: reject_both+=1
            if model_orig.pvalues[1]<0.05: reject_orig+=1
        outp[bdx,btx] = np.array((b1,boot,nsim,reject_orig,reject_boot,reject_both))
## Convert to Dataframe for analysis
df=pd.DataFrame(np.vstack(outp))
df.columns=['Betas','Bootstraps','No_of_Simulation','Reject_Actual','Reject_Bootstrap','Reject_Both']
df['Power of the Test(when Beta=0, its Size of Test)']=(df['Reject_Both']/df['No_of_Simulation'])*100


In [4]:
df

,Betas,Bootstraps,No_of_Simulation,Reject_Actual,Reject_Bootstrap,Reject_Both,"Power of the Test(when Beta=0, its Size of Test)"
0,-0.1,10.0,1000.0,159.0,186.0,108.0,10.8
1,-0.1,20.0,1000.0,153.0,100.0,71.0,7.1
2,-0.1,30.0,1000.0,183.0,56.0,52.0,5.2
3,-0.1,50.0,1000.0,181.0,97.0,87.0,8.7
4,0.0,10.0,1000.0,47.0,87.0,28.0,2.8
5,0.0,20.0,1000.0,42.0,27.0,17.0,1.7
6,0.0,30.0,1000.0,50.0,18.0,13.0,1.3
7,0.0,50.0,1000.0,51.0,17.0,14.0,1.4
8,0.1,10.0,1000.0,164.0,199.0,106.0,10.6
9,0.1,20.0,1000.0,159.0,90.0,69.0,6.9


In [ ]:
#estimation coud be improved by decreasing the nuber of bootstraps